In [5]:
import numpy as np
import xgboost as xgb 
import sklearn.metrics

In [6]:
Ndata          = 5;
dataSets       = ['CSV/1year.csv','CSV/2year.csv','CSV/3year.csv','CSV/4year.csv','CSV/5year.csv'];
resutSets      = ['R/CV1year.csv','R/CV2year.csv','R/CV3year.csv','R/CV4year.csv','R/CV5year.csv'];
fillingMissing = -99999;

In [7]:
param = {'bst:eta':0.02, 
         'silent':1, 
         'objective':'binary:logistic',
         'bst:max_depth':11,
         'base_score':0.5,
         'subsample':0.9,
         'bst:colsample_bytree':0.5,
         'scale_pos_weight':1,
         'min_child_weight':5}
plst = param.items()
plst +=  [('eval_metric', 'auc')] 
num_round = 200

In [8]:
for l in range(0,Ndata):
    data = np.genfromtxt(dataSets[l], delimiter = ',')
    data[np.isnan(data)] = fillingMissing;
        
    data[np.isnan(data)] = 0;
    N = np.shape(data)[0];
    
    Nrep      = 10;
    Nop       = 4;
    NNewf     = 60;
    Threshold = 10;
    np.set_printoptions(precision = 4)
    
    numCV = 10;     
    resultsGether = np.zeros((numCV,Nrep+2),dtype = float);
    
    cvId =  np.random.randint(numCV, size = N);
    for i in range(0,numCV):
        num_features = np.shape(data)[1];
        dataTrain    = data[cvId!= i,:];
        dataTest     = data[cvId == i,:];
        labelsTrain  = dataTrain[:,(num_features-1)];
        dataTrain    = dataTrain[:,0:num_features-1];
    
        labelsTest         = dataTest[:,(num_features-1)];
        dataTest           = dataTest[:,0:num_features-1];
        NTrain             = np.shape(dataTrain)[0];
        NTest              = np.shape(dataTest)[0];
        num_features       = np.shape(dataTrain)[1];
        dtrain             = xgb.DMatrix( dataTrain, label = labelsTrain, missing = fillingMissing)
        dtest              = xgb.DMatrix( dataTest, label = labelsTest, missing = fillingMissing)
        evallist           = [(dtest,'eval'), (dtrain,'train')]
        bst                = xgb.train( plst, dtrain, num_round, evallist,early_stopping_rounds = 10)
        resultsGether[i,0] = float(bst.eval(dtest, 'test',0).split(":")[1]);
        p                  = np.zeros((num_features),dtype = float)
        #featuresNew       = np.zeros((NNewf,3),dtype = float)
        p                  = p  + 1/float(num_features);
        p_test             = np.zeros((NTest,Nrep),dtype = float);
        for j in range(0,Nrep):
            #here draw
            num_features = np.shape(p)[0];
            print np.shape(p)
            for k in range(0,NNewf):
                typeOpr = np.random.randint(Nop, size = 1);
                '''f1 = np.zeros((num_features,1),dtype = int);
                f2 = np.zeros((num_features,1),dtype = int);'''
                f1 = np.random.multinomial(1, p, size = 1)[0]
                f2 = np.random.multinomial(1, p, size = 1)[0]
                '''print('type: ' + str(typeOpr))
                print('f1: ' + str(f1.ravel().nonzero()))
                print('f2: ' + str(f2.ravel().nonzero()))'''
                '''featuresNew[k,0] = typeOpr;
                featuresNew[k,1] = np.where(f1!= 0)[0][0];
                featuresNew[k,2] = np.where(f2!= 0)[0][0]; '''
                if(typeOpr == 0):
                    newColTrain = dataTrain[:, f1 == 1] +dataTrain[:,f2 == 1];  
                    newColTest = dataTest[:, f1 == 1] +dataTest[:,f2 == 1]; 
                elif(typeOpr == 1):
                    newColTrain = dataTrain[:, f1 == 1] - dataTrain[:,f2 == 1];  
                    newColTest = dataTest[:, f1 == 1] - dataTest[:,f2 == 1];
                elif(typeOpr == 2): 
                    newColTrain = dataTrain[:, f1 == 1] * dataTrain[:,f2 == 1];  
                    newColTest = dataTest[:, f1 == 1] * dataTest[:,f2 == 1];
                else:
                    newColTrain = dataTrain[:, f1 == 1] / dataTrain[:,f2 == 1];  
                    newColTest = dataTest[:, f1 == 1] / dataTest[:,f2 == 1]; 
                dataTrain = np.append(dataTrain,newColTrain, axis = 1);
                dataTest = np.append(dataTest,newColTest, axis = 1);
                p = np.append(p,[0], axis = 0);
           # np.savetxt("featuresNew.csv", featuresNew, delimiter = ",")
            print np.shape(dataTrain)
            print np.shape(dataTest)
            dataTrain[np.isnan(dataTrain)] = 0;
            dataTest[np.isnan(dataTest)] = 0;
            dtrain               = xgb.DMatrix( dataTrain, label = labelsTrain, missing = fillingMissing)
            dtest                = xgb.DMatrix( dataTest, label = labelsTest, missing = fillingMissing)
            evallist             = [(dtest,'eval'), (dtrain,'train')]
            bst                  = xgb.train( plst, dtrain, num_round, evallist,early_stopping_rounds = 10)
            resultsGether[i,j+1] = float(bst.eval(dtest, 'test',0).split(":")[1]);
            p_test[:,j]          = bst.predict(dtest);
            fscore               = bst.get_fscore(fmap = '');
            if(j == Nrep-1):
                p_mean = np.mean(p_test, axis = 1);
                auc = sklearn.metrics.roc_auc_score(labelsTest, p_mean);
                resultsGether[i,j+2] = auc;
            print(fscore)
            keys = fscore.keys()
            print(len(keys))
            newColTrain = np.zeros((NTrain,1),dtype = float);
            newColTest = np.zeros((NTest,1),dtype = float);
            
            key = int(keys[0][1:]);
            p = np.zeros((len(keys)),dtype = float)+1/float(len(keys));
            newColTrain[:,0]  = dataTrain[:,key];
            newColTest[:,0]  = dataTest[:,key];
            dataTrainNEW = newColTrain;
            dataTestNEW = newColTest;
            for m in range(1,len(keys)):
                key              = int(keys[m][1:]);
                newColTrain[:,0] = dataTrain[:,key];
                newColTest[:,0]  = dataTest[:,key];
                dataTrainNEW     = np.append(dataTrainNEW,newColTrain, axis = 1);
                dataTestNEW      = np.append(dataTestNEW,newColTest, axis = 1);
                if(float(fscore[keys[m]])>Threshold):
                    p[m] = float(fscore[keys[m]]);
                else:
                    p[m] = 0;
            dataTrain = dataTrainNEW;
            dataTest = dataTestNEW;
            p = p/np.sum(p); 
    
    np.savetxt(resutSets[l], resultsGether, delimiter = ",")

[0]	eval-auc:0.815777	train-auc:0.819935
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.857794	train-auc:0.896947
[2]	eval-auc:0.896828	train-auc:0.940802
[3]	eval-auc:0.891029	train-auc:0.944387
[4]	eval-auc:0.937626	train-auc:0.94189
[5]	eval-auc:0.940798	train-auc:0.954489
[6]	eval-auc:0.9425	train-auc:0.963965
[7]	eval-auc:0.940819	train-auc:0.973299
[8]	eval-auc:0.944496	train-auc:0.977557
[9]	eval-auc:0.948508	train-auc:0.980715
[10]	eval-auc:0.942542	train-auc:0.98227
[11]	eval-auc:0.94584	train-auc:0.987705
[12]	eval-auc:0.943761	train-auc:0.990029
[13]	eval-auc:0.948067	train-auc:0.992566
[14]	eval-auc:0.952899	train-auc:0.99376
[15]	eval-auc:0.948319	train-auc:0.995631
[16]	eval-auc:0.94605	train-auc:0.99608
[17]	eval-auc:0.951597	train-auc:0.997017
[18]	eval-auc:0.951261	train-auc:0.997606
[19]	eval-auc:0.953571	train-auc:0.997963
[20]	eval-auc:0.956134	train-auc:0.

C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in divide
C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in divide
C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in divide
C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in divide


(6312L, 124L)
(715L, 124L)
[0]	eval-auc:0.847962	train-auc:0.863266
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.884664	train-auc:0.885931
[2]	eval-auc:0.900882	train-auc:0.922362
[3]	eval-auc:0.901534	train-auc:0.929921
[4]	eval-auc:0.92292	train-auc:0.949463
[5]	eval-auc:0.920294	train-auc:0.949342
[6]	eval-auc:0.927983	train-auc:0.956461
[7]	eval-auc:0.931681	train-auc:0.965877
[8]	eval-auc:0.923487	train-auc:0.97119
[9]	eval-auc:0.933992	train-auc:0.976408
[10]	eval-auc:0.932038	train-auc:0.979335
[11]	eval-auc:0.937857	train-auc:0.983154
[12]	eval-auc:0.944076	train-auc:0.984766
[13]	eval-auc:0.946197	train-auc:0.988043
[14]	eval-auc:0.943971	train-auc:0.988839
[15]	eval-auc:0.937248	train-auc:0.990075
[16]	eval-auc:0.937668	train-auc:0.991687
[17]	eval-auc:0.941134	train-auc:0.993619
[18]	eval-auc:0.94542	train-auc:0.995454
[19]	eval-auc:0.94521	train-auc:0.996928
[20]

C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in subtract
C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in subtract
C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: invalid value encountered in multiply
C:\Users\15000\Anaconda2\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in multiply


(6312L, 170L)
(715L, 170L)
[0]	eval-auc:0.882101	train-auc:0.848368
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.900525	train-auc:0.885766
[2]	eval-auc:0.899433	train-auc:0.90047
[3]	eval-auc:0.897626	train-auc:0.903674
[4]	eval-auc:0.927038	train-auc:0.931287
[5]	eval-auc:0.927521	train-auc:0.941865
[6]	eval-auc:0.923929	train-auc:0.95641
[7]	eval-auc:0.929118	train-auc:0.9676
[8]	eval-auc:0.931092	train-auc:0.97366
[9]	eval-auc:0.938845	train-auc:0.982232
[10]	eval-auc:0.942101	train-auc:0.985559
[11]	eval-auc:0.945798	train-auc:0.987056
[12]	eval-auc:0.951723	train-auc:0.989582
[13]	eval-auc:0.951723	train-auc:0.991082
[14]	eval-auc:0.955189	train-auc:0.993244
[15]	eval-auc:0.952752	train-auc:0.993883
[16]	eval-auc:0.953592	train-auc:0.994871
[17]	eval-auc:0.955609	train-auc:0.994962
[18]	eval-auc:0.950693	train-auc:0.99601
[19]	eval-auc:0.955126	train-auc:0.996979
[20]	e

...

[2]	eval-auc:0.916129	train-auc:0.94439
[3]	eval-auc:0.91987	train-auc:0.960407
[4]	eval-auc:0.919734	train-auc:0.963198
[5]	eval-auc:0.921659	train-auc:0.967278
[6]	eval-auc:0.91884	train-auc:0.974533
[7]	eval-auc:0.930415	train-auc:0.980992
[8]	eval-auc:0.957007	train-auc:0.985318
[9]	eval-auc:0.957549	train-auc:0.986876
[10]	eval-auc:0.955544	train-auc:0.989157
[11]	eval-auc:0.96167	train-auc:0.990397
[12]	eval-auc:0.959881	train-auc:0.99191
[13]	eval-auc:0.963025	train-auc:0.993287
[14]	eval-auc:0.970561	train-auc:0.994431
[15]	eval-auc:0.968121	train-auc:0.995367
[16]	eval-auc:0.967525	train-auc:0.995866
[17]	eval-auc:0.970995	train-auc:0.996617
[18]	eval-auc:0.969206	train-auc:0.996885
[19]	eval-auc:0.968935	train-auc:0.99731
[20]	eval-auc:0.966495	train-auc:0.997791
[21]	eval-auc:0.968718	train-auc:0.998158
[22]	eval-auc:0.967308	train-auc:0.998397
[23]	eval-auc:0.967417	train-auc:0.998695
[24]	eval-auc:0.968772	train-auc:0.998811
[25]	eval-auc:0.968609	train-auc:0.99897
[26]	ev

(179L,)
(5348L, 239L)
(562L, 239L)
[0]	eval-auc:0.888262	train-auc:0.909635
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.899377	train-auc:0.9308
[2]	eval-auc:0.88997	train-auc:0.942577
[3]	eval-auc:0.918189	train-auc:0.956508
[4]	eval-auc:0.913879	train-auc:0.963134
[5]	eval-auc:0.921334	train-auc:0.972896
[6]	eval-auc:0.934237	train-auc:0.978626
[7]	eval-auc:0.938222	train-auc:0.980823
[8]	eval-auc:0.942261	train-auc:0.983281
[9]	eval-auc:0.941908	train-auc:0.985464
[10]	eval-auc:0.936568	train-auc:0.987548
[11]	eval-auc:0.933017	train-auc:0.989418
[12]	eval-auc:0.940092	train-auc:0.991047
[13]	eval-auc:0.937083	train-auc:0.992848
[14]	eval-auc:0.940689	train-auc:0.99454
[15]	eval-auc:0.94714	train-auc:0.99537
[16]	eval-auc:0.951369	train-auc:0.996434
[17]	eval-auc:0.952074	train-auc:0.996922
[18]	eval-auc:0.953592	train-auc:0.997419
[19]	eval-auc:0.95202	train-auc:0.997888

(202L,)
(5348L, 262L)
(562L, 262L)
[0]	eval-auc:0.884793	train-auc:0.910755
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.896042	train-auc:0.932676
[2]	eval-auc:0.918894	train-auc:0.95865
[3]	eval-auc:0.924261	train-auc:0.961949
[4]	eval-auc:0.93936	train-auc:0.96682
[5]	eval-auc:0.932936	train-auc:0.972676
[6]	eval-auc:0.93055	train-auc:0.982031
[7]	eval-auc:0.932665	train-auc:0.982412
[8]	eval-auc:0.935159	train-auc:0.984939
[9]	eval-auc:0.933966	train-auc:0.986708
[10]	eval-auc:0.931662	train-auc:0.989362
[11]	eval-auc:0.933641	train-auc:0.990871
[12]	eval-auc:0.937327	train-auc:0.992501
[13]	eval-auc:0.929954	train-auc:0.993816
[14]	eval-auc:0.929846	train-auc:0.994718
[15]	eval-auc:0.935159	train-auc:0.995763
[16]	eval-auc:0.938195	train-auc:0.99613
[17]	eval-auc:0.933207	train-auc:0.996913
[18]	eval-auc:0.938303	train-auc:0.997381
[19]	eval-auc:0.936189	train-auc:0.9978

(177L,)
(5348L, 237L)
(562L, 237L)
[0]	eval-auc:0.88444	train-auc:0.907221
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.884386	train-auc:0.930078
[2]	eval-auc:0.906479	train-auc:0.945574
[3]	eval-auc:0.903361	train-auc:0.962177
[4]	eval-auc:0.917593	train-auc:0.971627
[5]	eval-auc:0.926023	train-auc:0.974782
[6]	eval-auc:0.934779	train-auc:0.980946
[7]	eval-auc:0.947357	train-auc:0.986152
[8]	eval-auc:0.944944	train-auc:0.987271
[9]	eval-auc:0.947899	train-auc:0.989215
[10]	eval-auc:0.952562	train-auc:0.991407
[11]	eval-auc:0.953429	train-auc:0.993361
[12]	eval-auc:0.95202	train-auc:0.993976
[13]	eval-auc:0.95511	train-auc:0.995158
[14]	eval-auc:0.956736	train-auc:0.995687
[15]	eval-auc:0.95679	train-auc:0.99638
[16]	eval-auc:0.952941	train-auc:0.997344
[17]	eval-auc:0.951965	train-auc:0.997769
[18]	eval-auc:0.952941	train-auc:0.998127
[19]	eval-auc:0.955815	train-auc:0.9987

(170L,)
(5348L, 230L)
(562L, 230L)
[0]	eval-auc:0.885281	train-auc:0.907299
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 10 rounds.
[1]	eval-auc:0.903009	train-auc:0.925353
[2]	eval-auc:0.918514	train-auc:0.948735
[3]	eval-auc:0.916725	train-auc:0.955033
[4]	eval-auc:0.917891	train-auc:0.963519
[5]	eval-auc:0.928788	train-auc:0.972519
[6]	eval-auc:0.937137	train-auc:0.980353
[7]	eval-auc:0.943562	train-auc:0.984656
[8]	eval-auc:0.949336	train-auc:0.987675
[9]	eval-auc:0.948387	train-auc:0.988488
[10]	eval-auc:0.949905	train-auc:0.989904
[11]	eval-auc:0.953483	train-auc:0.991432
[12]	eval-auc:0.959122	train-auc:0.992975
[13]	eval-auc:0.966278	train-auc:0.994155
[14]	eval-auc:0.963676	train-auc:0.994953
[15]	eval-auc:0.964923	train-auc:0.995901
[16]	eval-auc:0.966766	train-auc:0.996627
[17]	eval-auc:0.966712	train-auc:0.997401
[18]	eval-auc:0.964489	train-auc:0.997833
[19]	eval-auc:0.964489	train-auc:0